### Youtube Sentence With Word And Order With Timestamp

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
from datetime import date
from datetime import time
from datetime import datetime
from datetime import timedelta

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
word_num = 300
df_word = pd.read_csv("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/Turkish/Word Tokenize Merge/Youtube_Word_Tokenize_Merge.csv") # word merge dataframe
df_word = df_word.head(word_num)
df_word

,word,frequency
0,bir,1173600
1,bu,840135
2,ve,590780
3,çok,497301
4,da,486717
...,...,...
295,hasta,15873
296,kötü,15853
297,birazcık,15834
298,geldim,15826


In [4]:
data_kind = "sentence"
df_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/Turkish/Sentence Clean Merge/Clean_Youtube_{data_kind.lower().capitalize()}_Merge_Result.csv")
df_sent

,start_time,end_time,sentence,video_id
0,00:00:06.680,00:00:07.340,sen kimsin peki,JR7nKXxUVtM
1,00:00:07.340,00:00:08.600,parolayı söyle,JR7nKXxUVtM
2,00:00:09.440,00:00:11.060,ya ben kim olacağım,JR7nKXxUVtM
3,00:00:11.060,00:00:11.860,keloğlan ım,JR7nKXxUVtM
4,00:00:13.380,00:00:13.880,ey,JR7nKXxUVtM
...,...,...,...,...
387903,00:02:11.720,00:02:14.440,gözlerin bir an bile çıkmadı aklımdan,7R3K6EDaBGE
387904,00:02:15.560,00:02:19.320,koynumdaysa hala o ufak bin lira,7R3K6EDaBGE
387905,00:02:20.320,00:02:21.200,sahi,7R3K6EDaBGE
387906,00:02:21.720,00:02:24.480,diğer eşi sende mi hala,7R3K6EDaBGE


In [5]:
#pd.to_datetime(df_sent['start_time'])

In [6]:
#df_sent['start_time'] = pd.to_datetime(df_sent['start_time']).dt.time

In [7]:
word_list = df_word.iloc[:,0].values.tolist()

In [8]:
sent_list  = df_sent.iloc[:,2].values.tolist()

In [9]:
resultlist = []

In [10]:
#import os
#from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
resultlist = manager.list()

def word_in_wordgroup(sent_list):
    mergelist = []
    try:
        word = sent_list.split()
    except:
        pass
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist.append(word[j])
            if len(mergelist) == len(word):
                    resultlist.append(sent_list)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, sent_list) # string_word liste

In [11]:
result_list = list(resultlist)
df_result = pd.DataFrame(result_list)
df_result.drop_duplicates(inplace=True)
df_result = df_result.rename(columns = {0: f"{data_kind.lower()}"})
df_sent_merge = pd.merge(df_result, df_sent, how="inner", on=f"{data_kind.lower()}")
df_sent_merge.drop_duplicates(inplace=True)
df_sent_with_words = df_sent_merge.sort_values(by=["video_id","start_time"], ascending=True)
df_sent_with_words.reset_index(drop=True, inplace=True)
df_sent_with_words

,sentence,start_time,end_time,video_id
0,yeni,00:00:02.304,00:00:02.816,0Bs9I3H1OJk
1,ne kadar,00:00:30.976,00:00:31.488,0Bs9I3H1OJk
2,bunun için,00:01:07.072,00:01:07.584,0Bs9I3H1OJk
3,şu anda,00:02:15.168,00:02:15.680,0Bs9I3H1OJk
4,birkaç gün içerisinde,00:02:34.112,00:02:34.624,0Bs9I3H1OJk
...,...,...,...,...
16550,çok güzel olmuş,00:02:33.583,00:02:35.008,zwWS6yifThY
16551,var,00:08:13.490,00:08:19.860,zzXTIIo7EvM
16552,ben,00:12:56.180,00:13:04.974,zzXTIIo7EvM
16553,burası da,00:22:55.450,00:23:01.260,zzXTIIo7EvM


In [12]:
#df_merge_result.to_csv(f"Youtube_{data_kind.capitalize()}_With_{word_num}_Words.csv", index=False)

In [13]:
df_sent_with_words["with_word"] = 1
df_sent_with_words

,sentence,start_time,end_time,video_id,with_word
0,yeni,00:00:02.304,00:00:02.816,0Bs9I3H1OJk,1
1,ne kadar,00:00:30.976,00:00:31.488,0Bs9I3H1OJk,1
2,bunun için,00:01:07.072,00:01:07.584,0Bs9I3H1OJk,1
3,şu anda,00:02:15.168,00:02:15.680,0Bs9I3H1OJk,1
4,birkaç gün içerisinde,00:02:34.112,00:02:34.624,0Bs9I3H1OJk,1
...,...,...,...,...,...
16550,çok güzel olmuş,00:02:33.583,00:02:35.008,zwWS6yifThY,1
16551,var,00:08:13.490,00:08:19.860,zzXTIIo7EvM,1
16552,ben,00:12:56.180,00:13:04.974,zzXTIIo7EvM,1
16553,burası da,00:22:55.450,00:23:01.260,zzXTIIo7EvM,1


In [14]:
df_with_word = pd.merge(df_sent, df_sent_with_words, how="left", on=["sentence","start_time","end_time","video_id"])
df_with_word.fillna(0, inplace=True)
df_with_word

,start_time,end_time,sentence,video_id,with_word
0,00:00:06.680,00:00:07.340,sen kimsin peki,JR7nKXxUVtM,0.0
1,00:00:07.340,00:00:08.600,parolayı söyle,JR7nKXxUVtM,0.0
2,00:00:09.440,00:00:11.060,ya ben kim olacağım,JR7nKXxUVtM,0.0
3,00:00:11.060,00:00:11.860,keloğlan ım,JR7nKXxUVtM,0.0
4,00:00:13.380,00:00:13.880,ey,JR7nKXxUVtM,0.0
...,...,...,...,...,...
387903,00:02:11.720,00:02:14.440,gözlerin bir an bile çıkmadı aklımdan,7R3K6EDaBGE,0.0
387904,00:02:15.560,00:02:19.320,koynumdaysa hala o ufak bin lira,7R3K6EDaBGE,0.0
387905,00:02:20.320,00:02:21.200,sahi,7R3K6EDaBGE,0.0
387906,00:02:21.720,00:02:24.480,diğer eşi sende mi hala,7R3K6EDaBGE,0.0


In [15]:
df_with_word.with_word.value_counts()

0.0    371353
1.0     16555
Name: with_word, dtype: int64

In [36]:
# # Time Duration
# df_time_duration = pd.DataFrame(df_with_word.groupby("video_id")["end_time"].max())
# df_time_duration.reset_index(inplace=True)
# df_time_duration

In [15]:
# First Code Running For All Sentence
# zeros = []
# ones = []
# zero_list = []
# one_list = []
# for i in range(len(df_with_word)):    
#     if df_with_word.loc[i,"with_word"] == 0:
#         zeros.append(i)
#         try:
#             one_list.append((ones[0],len(ones)))
#         except:
#             pass
#         ones = []
#     else:
#         ones.append(i)
#         try:
#             zero_list.append((zeros[0],len(zeros)))
#         except:
#             pass
#         zeros = []
# try:
#     one_list.append((ones[0],len(ones)))
# except:
#     pass
# try:
#     zero_list.append((zeros[0],len(zeros)))
# except:
#     pass# 

In [17]:
# Code Running For Video_id Sentence
zeros = []
ones = []
zero_list = []
one_list = []
for i in range(len(df_with_word)):
    videoid = df_with_word.loc[i,"video_id"]    
    if df_with_word.loc[i,"with_word"] == 0:
        zeros.append(i)
        try:
            one_list.append((ones[0],len(ones)))
        except:
            pass
        ones = []
    else:
        ones.append(i)
        try:
            zero_list.append((zeros[0],len(zeros)))
        except:
            pass
        zeros = []

    try:
        if df_with_word.loc[i+1,"video_id"] != videoid:
            try:
                one_list.append((ones[0],len(ones)))
                ones = []
            except:
                pass
            try:
                zero_list.append((zeros[0],len(zeros)))
                zeros = []
            except:
                pass
        else:
            pass
    except:
        pass

try:
    one_list.append((ones[0],len(ones)))
except:
    pass
try:
    zero_list.append((zeros[0],len(zeros)))
except:
    pass

In [18]:
len(one_list)

13996

In [19]:
len(zero_list)

18784

In [37]:
df_one = pd.DataFrame(one_list)
df_one.rename(columns={0:"index",1:"length"}, inplace=True)
df_one.drop_duplicates(inplace=True)
df_one.sort_values(by="length", ascending=False, inplace=True)
df_one.reset_index(drop=True, inplace=True)
df_one

,index,length
0,286907,39
1,39290,31
2,256862,29
3,145435,22
4,257157,17
...,...,...
13991,109976,1
13992,109983,1
13993,110066,1
13994,110236,1


In [38]:
df_one["length"].sum()

16555

In [41]:
df_one_all = pd.DataFrame()
for i in range(len(df_one)):
    index = df_one.loc[i,"index"]
    length = df_one.loc[i,"length"]
    start_index = index
    end_index = index+length
    df_video = df_with_word.iloc[start_index:end_index,]
    df_one_all = pd.concat([df_one_all, df_video],axis=0)
df_one_all.reset_index(inplace=True)
df_one_all

,index,start_time,end_time,sentence,video_id,with_word
0,286907,00:02:09.190,00:02:16.800,abone ol,jgOHc32Yrq0,1.0
1,286908,00:02:16.800,00:02:19.170,abone ol,jgOHc32Yrq0,1.0
2,286909,00:02:21.580,00:02:29.800,abone ol,jgOHc32Yrq0,1.0
3,286910,00:02:34.800,00:02:37.340,abone ol abone ol,jgOHc32Yrq0,1.0
4,286911,00:02:40.780,00:02:42.280,o kim ya,jgOHc32Yrq0,1.0
...,...,...,...,...,...,...
16550,109976,00:12:08.063,00:12:08.575,bu,VeHj0iEgu8A,1.0
16551,109983,00:12:36.479,00:12:38.783,ve bunu,VeHj0iEgu8A,1.0
16552,110066,00:00:07.496,00:00:08.765,merhaba arkadaşlar,ZCOpRdv8vXU,1.0
16553,110236,00:03:24.160,00:03:27.900,orada,pyB5v0sjvvg,1.0


In [44]:
df_word_result = pd.DataFrame()
for i in word_list:
    word_in_video = df_one_all[df_one_all.sentence.str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)] 
    word_in_video.insert(0,"word",i)
    df_word_result = pd.concat([df_word_result,word_in_video], axis=0)
df_word_result.reset_index(drop=True, inplace=True)
df_word_result = df_word_result.sort_values(by=["video_id","index"], ascending=False)
df_word_result.reset_index(drop=True, inplace=True)
df_word_result

,word,index,start_time,end_time,sentence,video_id,with_word
0,tam,93148,00:25:14.035,00:25:19.890,tam,zzXTIIo7EvM,1.0
1,da,93130,00:22:55.450,00:23:01.260,burası da,zzXTIIo7EvM,1.0
2,burası,93130,00:22:55.450,00:23:01.260,burası da,zzXTIIo7EvM,1.0
3,ben,93041,00:12:56.180,00:13:04.974,ben,zzXTIIo7EvM,1.0
4,var,92999,00:08:13.490,00:08:19.860,var,zzXTIIo7EvM,1.0
...,...,...,...,...,...,...,...
31847,için,361759,00:01:07.072,00:01:07.584,bunun için,0Bs9I3H1OJk,1.0
31848,bunun,361759,00:01:07.072,00:01:07.584,bunun için,0Bs9I3H1OJk,1.0
31849,ne,361740,00:00:30.976,00:00:31.488,ne kadar,0Bs9I3H1OJk,1.0
31850,kadar,361740,00:00:30.976,00:00:31.488,ne kadar,0Bs9I3H1OJk,1.0


In [45]:
df_word_nunique = pd.DataFrame(df_word_result.groupby("video_id")["word"].nunique())
df_word_nunique.reset_index(inplace=True)
df_word_nunique.rename(columns={"word":"word_num"}, inplace=True)
df_word_nunique.sort_values(by="word_num", inplace=True, ascending=False)
df_word_nunique

,video_id,word_num
1164,UDDfBEUGY60,93
2010,rXAAGktXqQ4,87
206,6FtA8_D7Ynw,80
1518,dD3A1IseY00,78
1733,k1ni27c9C2k,76
...,...,...
1332,YVi80j2UiVc,1
885,NH3W5hzAHUc,1
1907,ouzjipOkfHU,1
1904,opZiFQhTqio,1


In [48]:
df_word_num_merge = pd.merge(df_word_result, df_word_nunique, how="inner", on="video_id")
df_word_num_merge

,word,index,start_time,end_time,sentence,video_id,with_word,word_num
0,tam,93148,00:25:14.035,00:25:19.890,tam,zzXTIIo7EvM,1.0,5
1,da,93130,00:22:55.450,00:23:01.260,burası da,zzXTIIo7EvM,1.0,5
2,burası,93130,00:22:55.450,00:23:01.260,burası da,zzXTIIo7EvM,1.0,5
3,ben,93041,00:12:56.180,00:13:04.974,ben,zzXTIIo7EvM,1.0,5
4,var,92999,00:08:13.490,00:08:19.860,var,zzXTIIo7EvM,1.0,5
...,...,...,...,...,...,...,...,...
31847,için,361759,00:01:07.072,00:01:07.584,bunun için,0Bs9I3H1OJk,1.0,24
31848,bunun,361759,00:01:07.072,00:01:07.584,bunun için,0Bs9I3H1OJk,1.0,24
31849,ne,361740,00:00:30.976,00:00:31.488,ne kadar,0Bs9I3H1OJk,1.0,24
31850,kadar,361740,00:00:30.976,00:00:31.488,ne kadar,0Bs9I3H1OJk,1.0,24


In [50]:
df_word_num_length_merge = pd.merge(df_word_num_merge, df_one, how="inner", on="index")
df_word_num_length_merge

,word,index,start_time,end_time,sentence,video_id,with_word,word_num,length
0,tam,93148,00:25:14.035,00:25:19.890,tam,zzXTIIo7EvM,1.0,5,1
1,da,93130,00:22:55.450,00:23:01.260,burası da,zzXTIIo7EvM,1.0,5,1
2,burası,93130,00:22:55.450,00:23:01.260,burası da,zzXTIIo7EvM,1.0,5,1
3,ben,93041,00:12:56.180,00:13:04.974,ben,zzXTIIo7EvM,1.0,5,1
4,var,92999,00:08:13.490,00:08:19.860,var,zzXTIIo7EvM,1.0,5,1
...,...,...,...,...,...,...,...,...,...
27045,için,361759,00:01:07.072,00:01:07.584,bunun için,0Bs9I3H1OJk,1.0,24,1
27046,bunun,361759,00:01:07.072,00:01:07.584,bunun için,0Bs9I3H1OJk,1.0,24,1
27047,ne,361740,00:00:30.976,00:00:31.488,ne kadar,0Bs9I3H1OJk,1.0,24,1
27048,kadar,361740,00:00:30.976,00:00:31.488,ne kadar,0Bs9I3H1OJk,1.0,24,1


In [56]:
df_word_num_length_merge.sort_values(["index"], ascending=True, inplace=True)
df_word_num_length_merge

,word,index,start_time,end_time,sentence,video_id,with_word,word_num,length
26010,ne,59,00:00:15.780,00:00:16.600,ne,3jq75SKokNQ,1.0,4,1
26009,oğlum,65,00:00:30.620,00:00:32.120,oğlum,3jq75SKokNQ,1.0,4,1
26008,nasıl,81,00:01:35.920,00:01:37.000,nasıl yani,3jq75SKokNQ,1.0,4,1
26007,yani,81,00:01:35.920,00:01:37.000,nasıl yani,3jq75SKokNQ,1.0,4,1
3048,buraya,153,00:00:05.620,00:00:06.120,buraya,tCadoXCM31Y,1.0,2,1
...,...,...,...,...,...,...,...,...,...
20874,gece,387845,00:00:52.650,00:00:53.750,gece,Dyh9n6F9by0,1.0,1,1
20873,gece,387851,00:01:07.150,00:01:08.150,gece,Dyh9n6F9by0,1.0,1,1
20872,gece,387853,00:01:11.650,00:01:12.600,gece,Dyh9n6F9by0,1.0,1,1
20871,gece,387866,00:02:10.000,00:02:15.000,gece,Dyh9n6F9by0,1.0,1,1


In [22]:
# df_zero = pd.DataFrame(zero_list)
# df_zero.rename(columns={0:"index",1:"length"}, inplace=True)
# df_zero.drop_duplicates(inplace=True)
# df_zero.sort_values(by="length", ascending=False, inplace=True)
# df_zero.reset_index(drop=True, inplace=True)
# df_zero

In [23]:
# df_zero["length"].sum()

In [25]:
df_word_result = pd.DataFrame()
for i in word_list:
    word_in_video = df_sent_with_words[df_sent_with_words.sentence.str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)] # try and except will try for sample 
    word_in_video.insert(0,"word",i)
    df_word_result = pd.concat([df_word_result,word_in_video], axis=0)
df_word_result.reset_index(drop=True, inplace=True)
df_word_result = df_word_result.sort_values(by=["video_id","start_time"], ascending=True)
df_word_result.reset_index(drop=True, inplace=True)
df_word_result

,word,sentence,start_time,end_time,video_id,with_word
0,yeni,yeni,00:00:02.304,00:00:02.816,0Bs9I3H1OJk,1
1,ne,ne kadar,00:00:30.976,00:00:31.488,0Bs9I3H1OJk,1
2,kadar,ne kadar,00:00:30.976,00:00:31.488,0Bs9I3H1OJk,1
3,için,bunun için,00:01:07.072,00:01:07.584,0Bs9I3H1OJk,1
4,bunun,bunun için,00:01:07.072,00:01:07.584,0Bs9I3H1OJk,1
...,...,...,...,...,...,...
31847,var,var,00:08:13.490,00:08:19.860,zzXTIIo7EvM,1
31848,ben,ben,00:12:56.180,00:13:04.974,zzXTIIo7EvM,1
31849,da,burası da,00:22:55.450,00:23:01.260,zzXTIIo7EvM,1
31850,burası,burası da,00:22:55.450,00:23:01.260,zzXTIIo7EvM,1


In [26]:
#df_word_result.to_csv(f"Youtube_{data_kind.capitalize()}_With_{word_num}_Word_And_Words.csv", index=False)

In [27]:
df_group = pd.DataFrame(df_word_result.groupby("video_id")["word"].nunique())
df_group.reset_index(inplace=True)
df_group.rename(columns={"word":"frequency"}, inplace=True)
df_group.sort_values(by="frequency", ascending=False, inplace=True)
df_group.reset_index(drop=True, inplace=True)
df_group 

,video_id,frequency
0,UDDfBEUGY60,93
1,rXAAGktXqQ4,87
2,6FtA8_D7Ynw,80
3,dD3A1IseY00,78
4,k1ni27c9C2k,76
...,...,...
2338,YVi80j2UiVc,1
2339,NH3W5hzAHUc,1
2340,ouzjipOkfHU,1
2341,opZiFQhTqio,1


In [28]:
df_word_result[df_word_result.video_id == "rXAAGktXqQ4"]

,word,sentence,start_time,end_time,video_id,with_word
27492,çok,çok sağ ol,00:00:30.464,00:00:32.256,rXAAGktXqQ4,1
27493,ol,çok sağ ol,00:00:30.464,00:00:32.256,rXAAGktXqQ4,1
27494,sağ,çok sağ ol,00:00:30.464,00:00:32.256,rXAAGktXqQ4,1
27495,sana,sana geldi,00:01:49.056,00:01:50.592,rXAAGktXqQ4,1
27496,geldi,sana geldi,00:01:49.056,00:01:50.592,rXAAGktXqQ4,1
...,...,...,...,...,...,...
27613,ederim,teşekkür ederim,00:53:17.951,00:53:18.719,rXAAGktXqQ4,1
27614,bir,umarım burada bir,00:54:17.343,00:54:19.647,rXAAGktXqQ4,1
27615,burada,umarım burada bir,00:54:17.343,00:54:19.647,rXAAGktXqQ4,1
27616,umarım,umarım burada bir,00:54:17.343,00:54:19.647,rXAAGktXqQ4,1


In [29]:
df_with_word[df_with_word.video_id == "rXAAGktXqQ4"]

,start_time,end_time,sentence,video_id,with_word
188694,00:00:00.768,00:00:06.912,merhabalar sevgili alıyor yoldan dinleyenleri ...,rXAAGktXqQ4,0.0
188695,00:00:07.168,00:00:07.680,ailesi,rXAAGktXqQ4,0.0
188696,00:00:08.448,00:00:14.592,i̇nşallah mehmet ali akın özgörüş elini mustaf...,rXAAGktXqQ4,0.0
188697,00:00:14.848,00:00:16.384,bugün stüdyo konuğumuz,rXAAGktXqQ4,0.0
188698,00:00:16.640,00:00:19.456,ben her konuma ayrı ayrı öncelikle,rXAAGktXqQ4,0.0
...,...,...,...,...,...
189675,00:55:11.359,00:55:15.711,lütfen internete giriyorsunuz pusula yazıyorsu...,rXAAGktXqQ4,0.0
189676,00:55:15.967,00:55:18.527,mutlaka o muhteşem kredi izleyin derim,rXAAGktXqQ4,0.0
189677,00:55:18.783,00:55:20.319,dediğim gibi gerçekten beni çok etkili,rXAAGktXqQ4,0.0
189678,00:55:21.343,00:55:23.647,duyarlı olmanızı ötürü de bir kez daha,rXAAGktXqQ4,0.0
